In [19]:
# Code illustrating import of gluon library
!sudo pip install mxnet
from __future__ import division

# Silence warnings
import warnings
warnings.filterwarnings('ignore')

import mxnet as mx
from mxnet import gluon, autograd, ndarray
import numpy as np

%matplotlib inline
from __future__ import division
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats
from sklearn.model_selection import train_test_split
from collections import Counter

Let's decide on some graphic options. Since these data sets have so many columns, it might be nice to see more of them than the default.

In [3]:
pd.set_option('max_columns', 110)
plt.style.use('ggplot')

Here we're reading in our data files and saving them as variables.

In [40]:
persons = pd.read_csv('data/person.csv')
accidents = pd.read_csv('data/accident.csv')
vehicles = pd.read_csv('data/vehicle.csv', encoding ='latin1')  # In case you get encoding errors

accidents.head()

,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,MINUTE,NHS,RUR_URB,FUNC_SYS,RD_OWNER,ROUTE,TWAY_ID,TWAY_ID2,MILEPT,LATITUDE,LONGITUD,SP_JUR,HARM_EV,MAN_COLL,RELJCT1,RELJCT2,TYP_INT,WRK_ZONE,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,SCH_BUS,RAIL,NOT_HOUR,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR
0,1,10001,1,1,0,0,0,1,1,127,0,1,1,2015,5,2,40,0,1,3,1,3,SR-5,NaN,1754,33.878653,-87.325328,0,35,0,0,1,1,0,4,2,1,0,1,0,0000000,99,99,2,58,88,88,0,0,0,1,1
1,1,10002,1,1,0,0,0,1,1,83,0,1,1,2015,5,22,13,1,1,1,1,1,I-65,NaN,3604,34.910442,-86.908708,0,34,0,0,1,1,0,3,2,10,0,10,0,0000000,99,99,22,20,88,88,0,0,0,1,0
2,1,10003,1,1,0,0,0,2,2,11,0,1,1,2015,5,1,25,0,1,3,1,2,US-SR 6,NaN,1958,32.142006,-85.758456,0,42,0,0,1,1,0,4,2,1,0,1,0,0000000,99,99,1,45,99,99,0,0,0,1,1
3,1,10004,1,1,0,0,0,1,1,45,0,4,1,2015,1,0,57,0,1,4,1,3,SR-27,NaN,566,31.439814,-85.510300,0,53,0,0,1,1,0,4,2,10,0,10,0,0000000,99,99,1,15,88,88,0,0,0,1,1
4,1,10005,2,2,0,0,0,2,2,45,2050,7,1,2015,4,7,9,0,2,3,1,2,US-SR 53,HINTON WATERS AVE,308,31.319331,-85.515100,0,12,6,0,2,3,0,1,1,1,0,1,0,0000000,99,99,7,16,88,88,0,0,0,1,0


Let's get a baseline assessment of a model's ability to accurately classify an Accident without considering information contained within the Person or Vehicle tables.  Let's prepare our data for the Gluon multi-class classification model.

#Page 32 of FARS Data Manual
We will want to use the following features of the Accident table:
    

    'PEDS': Persons not in vehicles
    'PERNOTMVIT': Non-motorists in crash
    'VE_TOTAL': All vehicles in crash
    'VE_FORMS': Number of vehicles in transport
    'PVH_INVL': Nubmer of parked, working vehicles
    'PERSONS': Number of persons involved
    'PERMVIT': Number of motorists in accident
    'COUNTY': County where accident occured
    'CITY': City where accident occured
    'MONTH': Month of accident
    'DAY': Day of month of accident
    'DAY_WEEK': Day of week of accident
    'YEAR': Year of crash
    'HOUR': Hour of crash
    'MINUTE': Minute of hour of crash
    'TWAY_ID': Traffic Direction at time of crash
    'TWAY_ID2': Traffic Direction at time of crash
    # Not in data, but in data manual
    # 'CL_TWAY': Routing signal at time of accident
    'ROUTE': Routing signal at time of accident
    'SP_JUR': Special Jurisdiction
    'MILEPT' : Closest mile point
    'LATITUDE' : Latitute
    'LONGITUD': Longitude
    'TYP_INT': Type of intersection
    'REL_ROAD': Relation to Trafficway
    'C_M_ZONE': Work Zone
    'WRK_ZONE': Work Zone
    'LGT_COND': Light condition
    'WEATHER': Weather condition
    'WEATHE1': Weather condition
    'WEATHE2': Weather condition

    # Label
    'MAN_COLL': Manner of Collision
    'PMAN_COLL': Manner of Collision
    'HARM_EV': First injury or damage producing event in crash
    
    
*All features have scale/index discontinuities over the history of the data; this must be accounted for on a feature-by-feature basis

In [79]:
sub_features = [
    'PEDS',
    'PERNOTMVIT',
    'VE_TOTAL',
    'VE_FORMS',
    'PVH_INVL',
    'PERSONS',
    'PERMVIT',
    'COUNTY',
    'CITY',
    'MONTH',
    'DAY',
    'DAY_WEEK',
    'HOUR',
    #Following 2 features are the same over different time horizons
#     'TWAY_ID',
#     'TWAY_ID2',
    #Following 2 features are the same over different time horizons
#     'CL_TWAY',
    'ROUTE',
    'TYP_INT',
    'LGT_COND',
    #Following 3 features are the same over different time horizons
    'WEATHER',
#     'WEATHER1',
#     'WEATHER2'
    'DRUNK_DR',
    
    # LABELS: We can choose which to prediect
    # Manner of Collision
    'MAN_COLL'
    # First Harmul Event
#     'HARM_EV'
]

df_accidents = accidents[sub_features]
df_accidents.head()

,PEDS,PERNOTMVIT,VE_TOTAL,VE_FORMS,PVH_INVL,PERSONS,PERMVIT,COUNTY,CITY,MONTH,DAY,DAY_WEEK,HOUR,ROUTE,TYP_INT,LGT_COND,WEATHER,DRUNK_DR,MAN_COLL
0,0,0,1,1,0,1,1,127,0,1,1,5,2,3,1,2,1,1,0
1,0,0,1,1,0,1,1,83,0,1,1,5,22,1,1,2,10,0,0
2,0,0,1,1,0,2,2,11,0,1,1,5,1,2,1,2,1,1,0
3,0,0,1,1,0,1,1,45,0,1,4,1,0,3,1,2,10,1,0
4,0,0,2,2,0,2,2,45,2050,1,7,4,7,2,3,1,1,0,6


Now that we have a sparse dataframe with features and labels (HARM_EV), we'll want to encode our categorical feautures.  Before we do that, let's decrease some of the sparsity by removing features that are missing more than half of their values.

In [80]:
# Almost all instances of 0 map to None, so we will remove columns that have an abundance of them
def trim_features(data_frame, ratio, label):
    print 'Begining columns:', len(data_frame.columns)
    # columns not to apply trim_features to
    leave_out = [label, 'DRUNK_DR']
    rows = data_frame.shape[0]
    for col in data_frame.columns:
        non_zeros = data_frame[col].astype(bool).sum(axis=0)
        # If there are more zeros than a given ratio, drop column
        if col in leave_out:
            break
        elif (non_zeros/rows)<ratio:
            data_frame.drop([col], axis=1, inplace=True)
    
    print 'Ending columns:', len(data_frame.columns)
    return data_frame
        
    
# Trim geographical columns
def trim_geo(data_frame, columns):
    for col in columns:
        data_frame[col] = data_frame[col].apply(lambda x: 0 if x>9995 else x)
    return data_frame
        

# TODO: trim_geo on CITY too
df_accidents_trimmed = trim_geo(df_accidents, ['COUNTY'])
df_accidents_trimmed = trim_features(df_accidents, 0.5, 'MAN_COLL')

df_accidents_trimmed.head() 

Begining columns: 19
Ending columns: 15


,VE_TOTAL,VE_FORMS,PERSONS,PERMVIT,COUNTY,MONTH,DAY,DAY_WEEK,HOUR,ROUTE,TYP_INT,LGT_COND,WEATHER,DRUNK_DR,MAN_COLL
0,1,1,1,1,127,1,1,5,2,3,1,2,1,1,0
1,1,1,1,1,83,1,1,5,22,1,1,2,10,0,0
2,1,1,2,2,11,1,1,5,1,2,1,2,1,1,0
3,1,1,1,1,45,1,4,1,0,3,1,2,10,1,0
4,2,2,2,2,45,1,7,4,7,2,3,1,1,0,6


Now that we have semi-dense data, let's one-hot encode our cateogrical features

In [ ]:
sub_feature_classes = {
    'COUNTY': 1000,
    'CITY': 1000,
    'MONTH': 13,
    'DAY': 31,
    'DAY_WEEK': 7,
    'HOUR': 24,
    'ROUTE': 9,
    'TYP_INT': 10,
    'LGT_COND': 9,
    'WEATHER': 14,
    'DRUNK_DR': 2,
    
    'HARM_EV': 100,
    # LABEL
    'MAN_COLL': 100
}


# One-hot encode all categorical features within a dataframe
def one_hot_encode_matrix(feature_matrix, label):
    print feature_matrix
    def one_hot_encode(x, n_classes):
        return np.eye(n_classes)[x]

    # First we want to encode the label
    y = one_hot_encode(feature_matrix[label], sub_feature_classes[label])
    
    # Now encode input features
    for col in feature_matrix:
        if col in sub_feature_classes:
            x = one_hot_encode(feature_matrix[col], sub_feature_classes[col])
            feature_matrix[col] = x


one_hot_encode_matrix(df_accidents_trimmed, 'MAN_COLL')

       VE_TOTAL  VE_FORMS  PERSONS  PERMVIT  COUNTY  MONTH  DAY  DAY_WEEK  \
0             1         1        1        1     127      1    1         5   
1             1         1        1        1      83      1    1         5   
2             1         1        2        2      11      1    1         5   
3             1         1        1        1      45      1    4         1   
4             2         2        2        2      45      1    7         4   
5             1         1        2        2     111      1    8         5   
6             1         1        2        2      89      1    8         5   
7             1         1        1        1      73      1    3         7   
8             1         1        1        1     117      1   13         3   
9             2         2        2        2      33      1    5         2   
10            2         2        2        2      83      1    7         4   
11            3         3        6        6      95      1    9         6   